# Subspace Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

Generate some normally distributed data:

In [2]:
mean = np.array([-10, 10]) 
cov = np.array([[1, .5], [.5, 1]])
#cov = np.eye(2) 
data = np.random.multivariate_normal(mean, cov, size=1000)

x_range = (mean[0] - 5, mean[0] + 5)
y_range = (mean[1] - 5, mean[1] + 5)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=x_range,
    y_range=y_range
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='black')

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [31]:
model = beer.PPCA.create(torch.zeros(2).double(), 1, torch.eye(2).double())
#model = beer.NormalFullCovariance.create(torch.zeros(2).double(), torch.eye(2).double())

## Variational Bayes Training 

In [32]:
epochs = 100
lrate = 1
X = torch.from_numpy(data).double()
elbo_fn = beer.EvidenceLowerBound(len(X))
#params = [model.mean_param, model.subspace_param]
params = model.parameters
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
elbos = []
klds = []
for epoch in range(epochs):
    optimizer.zero_grad()
    elbo = elbo_fn(model, X)
    elbo.natural_backward()
    print(float(elbo._kl_div), float(elbo._exp_llh_per_frame.sum()))
    optimizer.step()
    elbos.append(round(float(elbo) / len(X), 5))
fig = figure(width=400, height=400)
fig.line(range(len(elbos)), elbos)
show(fig)

0.0 -80257.1258870317
85.81371048776873 -4837.4218032285935
89.62377581172976 -2962.2413873513406
110.95346818006851 -1600.9852321941999
118.07106256533417 -1264.6103330131866
120.86580836975517 -1252.3459478750506
121.05208653136745 -1252.4234033154667
121.05395404902123 -1254.2681262822125
121.03270087909239 -1258.661337224236
121.00459061226036 -1268.2965426496419
120.97694059010037 -1288.2510985218953
120.96755977453313 -1325.6279607253048
121.01094213477216 -1384.798526221972
121.15084631198445 -1459.5079348924694
121.40950859363528 -1534.9195536956802
121.75603212124037 -1599.6680468162865
122.11980986662195 -1650.1937452368334
122.43910922095387 -1687.5004780147906
122.6863905683391 -1714.0919747119958
122.8624603967437 -1732.5975331925592
122.9813569596771 -1745.2734793134434
123.05924516317134 -1753.870786788906
123.10951213235205 -1759.6678389906997
123.14177540595767 -1763.5639969412782
123.16246886573231 -1766.1783475674301
123.17574954888062 -1767.931761480271
123.18426646

In [5]:
model.precision, model.mean, model.subspace

AttributeError: 'NormalFullCovariance' object has no attribute 'precision'

In [ ]:
mean = model.mean.numpy()
evectors = model.subspace.numpy().T
print('bases:', evectors, evectors.shape)

fig1 = figure(
    title='PPCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig1.circle(data[:, 0], data[:, 1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 0])
alpha = 1
fig1.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)
alpha = np.linalg.norm(evectors[:, 1])
alpha = 1
fig1.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 1] + mean[0], 
        y_end=evectors[1, 1] + mean[1])
)

X = data
C = cov
evals, evectors = np.linalg.eigh(C)
print('evectors')
print(evectors)

fig2 = figure(
    title='PCA',
    x_range=x_range,
    y_range=y_range,
    width=400,
    height=400
)

fig2.circle(data[0], data[1], alpha=.3)

# Plot subspace.
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], 
        y_start=mean[1], 
        x_end=evectors[0, 0] + mean[0], 
        y_end=evectors[1, 0] + mean[1])
)
alpha = np.linalg.norm(evectors[:, 1])
fig2.add_layout(
    Arrow(
        end=NormalHead(size=10, line_alpha=alpha, fill_alpha=alpha),
        line_width=2,
        line_alpha=alpha,
        x_start=mean[0], y_start=mean[1], 
        x_end=evectors[0, 1] + mean[0], 
        y_end=evectors[1, 1] + mean[1])
)

grid = gridplot([[fig1, fig2]])
show(grid)

In [ ]:
torch.lgamma

In [ ]:
from scipy.special import gammaln as lgamma
np.isclose(lgamma(10000000), torch.lgamma(torch.tensor(10000000.)).numpy())

In [ ]:
_, _, _, _, m_quad, m_mean = model._get_expectation()
m_quad, m_mean = m_quad.numpy(), m_mean.numpy()

In [ ]:
m_quad, m_quad - np.sum(m_mean**2)

In [ ]:
m_mean, m_quad

In [ ]:
model.mean_param.expected_value()